<center>

**CRIPTOGRAFÍA DE LLAVE PÚBLICA**

</center>

<p align="center">
    <img src="https://logowik.com/content/uploads/images/escudo-de-la-universidad-nacional-de-colombia-20163327.logowik.com.webp" width="400">
</p>

# **🖋️Firma Digital RSA✒️**

<p align="center">
    <img src="https://media.istockphoto.com/id/1186072427/es/vector/icono-de-firma-electr%C3%B3nica.jpg?s=612x612&w=0&k=20&c=1o2ySIZwHQLcI0oPsBWmd-D75LPUlj0VUOL6Mc5zzQQ="width="400">
</p>



- **Generación de claves RSA**  
   - Se selecciona el tamaño de clave (1024, 2048, 3072 o 4096 bits).  
   - Se eligen dos primos grandes $p$ y $q$ y se calcula  
     $$
       N = p \times q,\quad
       \varphi(N) = (p-1)(q-1).
     $$  
   - Se escoge el exponente público $e$ (por ejemplo, 65537) con $\gcd(e,\varphi(N))=1$.  
   - Se obtiene el exponente privado  
     $$
       d = e^{-1} \bmod \varphi(N).
     $$  
   - La clave pública es $(N,e)$, la clave privada es $(N,d)$, ambas exportadas en formato PEM.

- **Firma de un mensaje**  
   - Se introduce el texto libre y se calcula su hash SHA-256:  
     $$
       H = \mathrm{SHA256}(\text{mensaje}).
     $$  
   - Se genera la firma usando PKCS#1 v1.5:  
     $$
       S = H^d \bmod N.
     $$  
   - El resultado se codifica en **Base64** para su transmisión.

- **Verificación de firma**  
   - Se recibe la firma en Base64 y se decodifica a bytes.  
   - Se vuelve a calcular  
     $$
       H' = \mathrm{SHA256}(\text{mensaje}).
     $$  
   - Se comprueba la igualdad  
     $$
       S^e \bmod N \;=\; H'.
     $$  
   - Si se cumple, la firma es válida; si no, no válida.



**📬Instalar paquetes📥**

In [9]:

!pip install pycryptodome --quiet

**📥Importaciones📦**

In [10]:

import hashlib
import base64
import ipywidgets as wd
from IPython.display import display, clear_output
from Crypto.PublicKey import RSA
from Crypto.Signature import pkcs1_15
from Crypto.Hash import SHA256

**👨‍💻Implementación👩‍💻**

In [11]:

def make_randfunc(passphrase: str):

    seed = hashlib.sha256(passphrase.encode('utf-8')).digest()
    counter = 0

    def randfunc(n: int) -> bytes:
        nonlocal counter
        output = b''
        while len(output) < n:
            data = seed + counter.to_bytes(8, 'big')
            digest = hashlib.sha256(data).digest()
            output += digest
            counter += 1
        return output[:n]

    return randfunc


rsa_key = None




txt_seed = wd.Text(
    description='Frase semilla:',
    placeholder='Escribe un texto (p.ej. "pepino")…',
    layout=wd.Layout(width='100%'),
    style={'description_width': '120px'}
)

key_size = wd.Dropdown(
    options=[1024, 2048, 3072, 4096],
    value=2048,
    description='Tamaño (bits):',
    style={'description_width': '120px'}
)

btn_gen = wd.Button(
    description='Generar par RSA',
    button_style='primary',
    icon='key',
    disabled=True
)

out_keys = wd.HTML(layout=wd.Layout(border='1px solid #ccc', padding='10px'))


msg_input = wd.Textarea(
    placeholder='Escribe el mensaje a firmar…',
    description='Mensaje:',
    layout=wd.Layout(width='100%', height='80px'),
    style={'description_width': '80px'}
)
btn_sign = wd.Button(
    description='Firmar mensaje',
    button_style='info',
    icon='pencil-alt'
)
out_sign = wd.HTML(layout=wd.Layout(border='1px solid #ccc', padding='10px'))


sig_input = wd.Textarea(
    placeholder='Pega la firma Base64 aquí…',
    description='Firma:',
    layout=wd.Layout(width='100%', height='80px'),
    style={'description_width': '80px'}
)
btn_verify = wd.Button(
    description='Verificar firma',
    button_style='success',
    icon='check'
)
out_ver = wd.HTML(layout=wd.Layout(border='1px solid #ccc', padding='10px'))


out_log = wd.HTML()

**👨‍💻Implementación👩‍💻**

**👨‍💻Implementación👩‍💻**

**👨‍💻Implementación👩‍💻**

**👨‍💻Implementación👩‍💻**

In [12]:

def on_seed_change(change):
    btn_gen.disabled = not bool(change['new'].strip())

txt_seed.observe(on_seed_change, names='value')


def on_generate(_):
    global rsa_key
    clear_output(wait=True)
    seed = txt_seed.value.strip()
    randfunc = make_randfunc(seed)
    # Generamos clave determinística
    rsa_key = RSA.generate(key_size.value, randfunc=randfunc)
    pub_pem  = rsa_key.publickey().export_key().decode('utf-8')
    priv_pem = rsa_key.export_key().decode('utf-8')
    out_keys.value = f"""
    <h4>Claves RSA ({key_size.value} bits)</h4>
    <b>Clave pública (PEM):</b>
    <pre style='font-size:12px'>{pub_pem}</pre>
    <b>Clave privada (PEM):</b>
    <pre style='font-size:12px'>{priv_pem}</pre>
    """
    out_log.value = "<b>✔ Claves generadas a partir de tu frase semilla.</b>"
    display(ui_sig)

def on_sign(_):
    global signature_b64
    clear_output(wait=True)
    if rsa_key is None:
        out_sign.value = "<span style='color:red;'>❗ Genera primero las claves RSA.</span>"
    else:
        h = SHA256.new(msg_input.value.encode('utf-8'))
        signer = pkcs1_15.new(rsa_key)
        sig = signer.sign(h)
        signature_b64 = base64.b64encode(sig).decode('ascii')
        out_sign.value = f"""
        <h4>Firma digital</h4>
        <b>Mensaje:</b> {msg_input.value}<br>
        <b>SHA-256:</b> {h.hexdigest()}<br>
        <b>Firma (Base64):</b>
        <pre style='font-size:12px'>{signature_b64}</pre>
        """
    display(ui_sig)

def on_verify(_):
    clear_output(wait=True)
    if rsa_key is None:
        out_ver.value = "<span style='color:red;'>❗ Genera primero las claves RSA.</span>"
    else:
        try:
            sig = base64.b64decode(sig_input.value)
            h = SHA256.new(msg_input.value.encode('utf-8'))
            verifier = pkcs1_15.new(rsa_key.publickey())
            verifier.verify(h, sig)
            out_ver.value = "<span style='color:green;'><b>✅ Firma válida.</b></span>"
        except (ValueError, TypeError):
            out_ver.value = "<span style='color:red;'><b>❌ Firma NO válida.</b></span>"
    display(ui_sig)


btn_gen.on_click(on_generate)
btn_sign.on_click(on_sign)
btn_verify.on_click(on_verify)


header = wd.HTML("<h2 style='color:#663399'>Módulo de Firma Digital RSA (Determinístico)</h2>")

ui_sig = wd.VBox([
    header,
    wd.HTML("<b>Paso 1:</b> introduce tu frase semilla y tamaño de clave"),
    wd.HBox([txt_seed, key_size, btn_gen]),
    out_keys,
    wd.HTML("<hr><b>Paso 2:</b> firma tu mensaje"),
    msg_input,
    btn_sign,
    out_sign,
    wd.HTML("<hr><b>Paso 3:</b> verifica la firma"),
    sig_input,
    btn_verify,
    out_ver,
    out_log
])

display(ui_sig)
